In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.optimize import curve_fit
from scipy.interpolate import CubicSpline, interp1d
from video_test import extract_frames
from camera_calibration import getCameraCalibrationValues
from utils import getPointClouds
from mpl_toolkits.mplot3d import Axes3D
import os


In [3]:
class Colors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'  # Resets the color to default


In [4]:
def getSobelXGradient(frame):
    sobelX = cv2.Sobel(frame, cv2.CV_64F, 1, 0)
    return sobelX

def getIntensityIndexOuterEdge(y_values):
    # print("Y values")
    # print(abs(y_values))
    x_values = np.arange(len(y_values))

    # Define the Gaussian function
    def gaussian(x, amplitude, mean, stddev):
        return amplitude * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))

    # Initial guess for the parameters: amplitude, mean, stddev
    initial_guess = [max(y_values), np.argmax(y_values), 1]

    # Perform the curve fitting
    try:
        params, covariance = curve_fit(gaussian, x_values, y_values, p0=initial_guess, maxfev=10000)
    except RuntimeError as e:
        print(f"{Colors.FAIL}Curve fitting failed: {e}{Colors.ENDC}")
        return np.argmax(y_values)

    # Extract the fitted parameters
    fitted_amplitude, fitted_mean, fitted_stddev = params

    # Find the index closest to the mean
    closest_index = np.argmin(np.round(np.abs(x_values - fitted_mean)))
    # print("Closest index to the mean:", closest_index)

    # Plot the original data and the fitted Gaussian curve
    # plt.plot(x_values, y_values, 'b-', label='data')
    # plt.plot(x_values, gaussian(x_values, *params), 'r--', label='fit: mean=%5.3f' % fitted_mean)
    # plt.axvline(x=fitted_mean, color='g', linestyle='--', label='Mean (µ)')
    # plt.axvline(x=closest_index, color='m', linestyle='--', label='Closest Index')
    # plt.xlabel('Index')
    # plt.ylabel('Y-values')
    # plt.legend()
    # plt.show()
    return closest_index
def getIntensityIndexInnerEdge(y_values):
    y_values = -(y_values)
    # print("Y values")
    # print(abs(y_values))
    x_values = np.arange(len(y_values))

    # Define the Gaussian function
    def gaussian(x, amplitude, mean, stddev):
        return amplitude * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))

    # Initial guess for the parameters: amplitude, mean, stddev
    initial_guess = [max(y_values), np.argmax(y_values), 1]

    # Perform the curve fitting
    try:
        params, covariance = curve_fit(gaussian, x_values, y_values, p0=initial_guess, maxfev=10000)
    except RuntimeError as e:
        print(f"{Colors.FAIL}Curve fitting failed: {e}{Colors.ENDC}")
        return np.argmax(y_values)

    # Extract the fitted parameters
    fitted_amplitude, fitted_mean, fitted_stddev = params

    # Find the index closest to the mean
    closest_index = np.argmin(np.round(np.abs(x_values - fitted_mean)))
    # print("Closest index to the mean:", closest_index)

    # Plot the original data and the fitted Gaussian curve
    # plt.plot(x_values, y_values, 'b-', label='data')
    # plt.plot(x_values, gaussian(x_values, *params), 'r--', label='fit: mean=%5.3f' % fitted_mean)
    # plt.axvline(x=fitted_mean, color='g', linestyle='--', label='Mean (µ)')
    # plt.axvline(x=closest_index, color='m', linestyle='--', label='Closest Index')
    # plt.xlabel('Index')
    # plt.ylabel('Y-values')
    # plt.legend()
    # plt.show()
    return closest_index



In [6]:
# def split_groups_polynomial(values, threshold, degree=3):
#     sorted_values = sorted(values)
#     # sorted_values = values
    
#     # Find the split point for the highest group
#     split_index = len(sorted_values)
#     # for i in range(1, len(sorted_values)):
#     #     diff = (sorted_values[i] - sorted_values[i - 1])
#     #     print(f"{Colors.OKBLUE}Difference between {sorted_values[i]} - {sorted_values[i-1]}{Colors.ENDC}: {diff}")
#     #     if sorted_values[i] - sorted_values[i - 1] > threshold:
#     #         split_index = i
#     #         break
#     for i in range(len(sorted_values) - 1, 0, -1):
#         # print(f"{Colors.OKBLUE}Difference between {Colors.ENDC}")
#         # print((sorted_values[i] - sorted_values[i - 1]))
#         if sorted_values[i] - sorted_values[i - 1] > threshold:
#             split_index = i
#             break
    
#     high_values = sorted_values[split_index:]
    
#     group1 = []
#     group2 = []
#     high_values_set = set(high_values)
#     used_high_values = set()
    
#     for value in values:
#         if value in high_values_set:
#             group1.append(value)
#             group2.append(None)
#             used_high_values.add(value)
#         else:
#             group1.append(None)
#             group2.append(value)
    
#     known_indices = [i for i, x in enumerate(group1) if x is not None]
#     known_values = [x for x in group1 if x is not None]
    
#     poly_interp = np.poly1d(np.polyfit(known_indices, known_values, degree))
#     interp_values = np.round(poly_interp(range(len(group1)))).astype(int)
    
#     group1 = [interp_values[i] if x is None else x for i, x in enumerate(group1)]
    
#     return group1, group2

In [5]:
def split_groups_polynomial(values, threshold, degree=3):
    # sorted_values = sorted(values)
    # # sorted_values = values
    
    # # Find the split point for the highest group
    # split_index = len(sorted_values)
    # # for i in range(1, len(sorted_values)):
    # #     diff = (sorted_values[i] - sorted_values[i - 1])
    # #     print(f"{Colors.OKBLUE}Difference between {sorted_values[i]} - {sorted_values[i-1]}{Colors.ENDC}: {diff}")
    # #     if sorted_values[i] - sorted_values[i - 1] > threshold:
    # #         split_index = i
    # #         break
    # for i in range(len(sorted_values) - 1, 0, -1):
    #     # print(f"{Colors.OKBLUE}Difference between {Colors.ENDC}")
    #     # print((sorted_values[i] - sorted_values[i - 1]))
    #     if sorted_values[i] - sorted_values[i - 1] > threshold:
    #         split_index = i
    #         break
    
    # high_values = sorted_values[split_index:]
    
    high_values = []
    maxReference = np.max(values)
    for i in values:
        if i >= (maxReference - threshold):
            high_values.append(i)
            
    group1 = []
    group2 = []
    high_values_set = set(high_values)
    used_high_values = set()
    
    for value in values:
        if value in high_values_set:
            group1.append(value)
            group2.append(None)
            used_high_values.add(value)
        else:
            group1.append(None)
            group2.append(value)
    
    known_indices = [i for i, x in enumerate(group1) if x is not None]
    known_values = [x for x in group1 if x is not None]
    
    poly_interp = np.poly1d(np.polyfit(known_indices, known_values, degree))
    interp_values = np.round(poly_interp(range(len(group1)))).astype(int)
    
    group1 = [interp_values[i] if x is None else x for i, x in enumerate(group1)]
    
    return group1, group2

In [6]:
def getDepthMap(frame, referenceLine, heightValues):
    depthMap = np.zeros(frame.shape)
    for i in range(frame.shape[0]):
        # print(f"{Colors.OKBLUE}({i}, {referenceLine[i]}, {heightValues[i]})")
        depthMap[i][int(referenceLine[i])] = heightValues[i]
    return depthMap

In [7]:
pixels_per_mm, cameraMatrix, dist, rvecs, tvecs = getCameraCalibrationValues()

Chessboard not found
1 mm = 10.627679228349166 pixels
Camera calibrated : 
 0.34352429263843676
Camera Matrix : 
 [[6.29530966e+03 0.00000000e+00 3.31140037e+02]
 [0.00000000e+00 6.36133646e+03 2.73235766e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 
 [[ 1.91572262e+01 -6.30755414e+03  2.78251225e-02  1.09198287e-02
  -2.47159800e+01]]
Rotation Vectors : 
 (array([[ 0.13408516],
       [-0.02418959],
       [-0.0025449 ]]), array([[-0.14921689],
       [ 0.07300657],
       [ 0.63636061]]), array([[-0.14169744],
       [ 0.08726113],
       [ 1.24802589]]), array([[-0.13515006],
       [ 0.08980708],
       [ 1.56805538]]), array([[0.12102242],
       [0.02120187],
       [0.46410927]]), array([[ 0.12186551],
       [-0.05546391],
       [ 0.98851071]]), array([[ 0.10437124],
       [-0.10527939],
       [ 1.34445345]]), array([[ 0.10135235],
       [-0.10412304],
       [ 1.49746484]]), array([[-0.13640589],
       [ 0.06773318],
       [ 0.62056035]]), array([[ 0.1136

In [28]:
# REFERENCE FRAME
objectName = "reference_03"
objectFrames = extract_frames(objectName, ".mp4")

mean_reference = []
count = 0

# Loop through the frames
for frame in objectFrames:
    # Get the Sobel X gradient
    # print(f"Processing frame {frame.shape}...")
    edgeTrace = getSobelXGradient(frame)

    # Calculate intensity index
    intensityInnerEdge = np.round(np.array([getIntensityIndexInnerEdge(i) for i in edgeTrace]))
    intensityOuterEdge = np.round(np.array([getIntensityIndexOuterEdge(i) for i in edgeTrace]))
    meanIndex = np.round((intensityInnerEdge + intensityOuterEdge) / 2)
    # print("Intensity Indices:")
    # print(intensity)

    # Convert the Sobel gradient image to absolute values to fit into the 8-bit range
    edgeTraceAbs = np.abs(edgeTrace).astype(np.uint8)
    
    # Convert the grayscale edgeTrace to BGR color image
    edgeTraceColor = cv2.cvtColor(edgeTraceAbs, cv2.COLOR_GRAY2BGR)

    # Define the output folder and create it if it doesn't exist
    output_folder = f'./edge_trace/{objectName}/'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Plot the intensity points on the edgeTraceColor image
    for idx, val in enumerate(intensityInnerEdge):
        cv2.circle(edgeTraceColor, (int(val), int(idx)), 1, (255, 0, 0), -1)  # Blue dot with radius 1

    for idx, val in enumerate(intensityOuterEdge):
        cv2.circle(edgeTraceColor, (int(val), int(idx)), 1, (0, 0, 255), -1)  # Red dot with radius 1

    for idx, val in enumerate(meanIndex):
        cv2.circle(edgeTraceColor, (int(val), int(idx)), 1, (0, 255, 255), -1)  # Red dot with radius 1

    # Save the image with the plotted points
    cv2.imwrite(f"./edge_trace/{objectName}/frame{count}.jpg", edgeTraceColor)
    mean_reference.append(abs(np.max(meanIndex) - np.min(meanIndex)))
    
    count += 1
    # break  # Remove this break to process all frames

# mean_reference = np.mean(mean_reference, axis=0)
# print(f"{Colors.OKGREEN}Mean Reference Indices:{Colors.ENDC}")
# print(mean_reference)
print(f"{Colors.OKGREEN}Deva of Reference Indices:{Colors.ENDC}")
print(mean_reference)

Extracted 71 frames from the video.
Deva of Reference Indices:
[3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]


In [8]:
def getPointClouds(depthMap, cameraIntrinsic):
    height, width = depthMap.shape
    points = []

    fx, fy = cameraIntrinsic[0, 0], cameraIntrinsic[1, 1]
    cx, cy = cameraIntrinsic[0, 2], cameraIntrinsic[1, 2]

    for v in range(height):
        for u in range(width):
            Z = depthMap[v, u]
            if Z == 0:  # Skip invalid depth values
                continue
            X = (u - cx) * Z / fx
            Y = (v - cy) * Z / fy
            points.append([X, Y, Z])

    points = np.array(points)
    print(f"{Colors.FAIL}Points:{Colors.ENDC}")
    print(points)
    
    # Create point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    return pcd

In [21]:
#OBJECT HEIGHT MEASUREMENT

# Define the object name and extract frames
objectName = "grey_step_3.68mm"
objectFrames = extract_frames(objectName, ".mp4")

meanPixelDisplacement = []
count = 0

heightList = []
all_point_clouds = []

# Define the translation distance between frames in mm
translation_distance = 0.75

# Loop through the frames
for frame in objectFrames:
    # Get the Sobel X gradient
    print(f"Processing frame {frame.shape}...")
    edgeTrace = getSobelXGradient(frame)

    # Calculate intensity index
    intensityInnerEdge = np.round(np.array([getIntensityIndexInnerEdge(i) for i in edgeTrace]))
    intensityOuterEdge = np.round(np.array([getIntensityIndexOuterEdge(i) for i in edgeTrace]))
    meanIndex = np.round((intensityInnerEdge + intensityOuterEdge) / 2)
    # print("Intensity Indices:")
    # print(intensity)

    # Convert the Sobel gradient image to absolute values to fit into the 8-bit range
    edgeTraceAbs = np.abs(edgeTrace).astype(np.uint8)
    
    # Convert the grayscale edgeTrace to BGR color image
    edgeTraceColor = cv2.cvtColor(edgeTraceAbs, cv2.COLOR_GRAY2BGR)

    # Define the output folder and create it if it doesn't exist
    output_folder = f'./edge_trace/{objectName}/'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Plot the intensity points on the edgeTraceColor image
    # for idx, val in enumerate(intensityInnerEdge):
    #     cv2.circle(edgeTraceColor, (int(val), int(idx)), 1, (255, 0, 0), -1)  # Blue dot with radius 1

    # for idx, val in enumerate(intensityOuterEdge):
    #     cv2.circle(edgeTraceColor, (int(val), int(idx)), 1, (0, 0, 255), -1)  # Red dot with radius 1

    # for idx, val in enumerate(meanIndex):
    #     cv2.circle(edgeTraceColor, (int(val), int(idx)), 1, (0, 255, 255), -1)  # Yellow dot with radius 1
    
    # print(f"{Colors.OKGREEN}Mean Index:{Colors.ENDC}")
    # print(meanIndex)
    c = 10.6276792
    referenceLine, _ = split_groups_polynomial(meanIndex, c/2)
    # print(f"{Colors.OKGREEN}Reconstructed reference:{Colors.ENDC}")
    # print(referenceLine)

    # for idx, val in enumerate(referenceLine):
    #     cv2.circle(edgeTraceColor, (int(val), int(idx)), 1, (0, 255, 0), -1)  # Green dot with radius 1
    
    # Save the image with the plotted points
    # cv2.imwrite(f"./edge_trace/{objectName}/frame{count}.jpg", edgeTraceColor)
    
    pixelDisplacement = abs(meanIndex - referenceLine)
    c = pixels_per_mm
    s = pixelDisplacement / c
    heights = 80 * s / (37.3 + s)

    depthMap = getDepthMap(frame, referenceLine, heights)

    output_folder = f'./depth_maps/{objectName}/'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Create a figure and axis
    fig, ax = plt.subplots()
    # Display the depth map
    cax = ax.imshow(depthMap)
    # Add a color bar
    fig.colorbar(cax, ax=ax)
    # Save the figure
    plt.savefig(f"{output_folder}/frame{count}.jpg")
    # Close the figure to free up memory
    plt.close(fig)

    pcd = getPointClouds(depthMap, cameraMatrix)

    # Translate the point cloud along the X-axis by the cumulative distance
    translation = np.array([count * translation_distance, 0, 0])
    pcd.translate(translation)

    # Add the translated point cloud to the list
    all_point_clouds.append(pcd)

    # heights = [value for value in heights if value > 0]
    # heightList.append(heights)

    
    
    

    count += 1
    # break  # Remove this break to process all frames

# print(f"{Colors.OKGREEN}Heights:{Colors.ENDC}")
# print(heights)
# print(f"{Colors.OKGREEN}Mean Heights:{Colors.ENDC}")
# print(f'{np.mean(heights)}')


combined_pcd = all_point_clouds[0]
for pcd in all_point_clouds[1:]:
    combined_pcd += pcd

# Save or visualize the combined point cloud
combined_pcd_folder = f'./combined_point_clouds/'
if not os.path.exists(combined_pcd_folder):
    os.makedirs(combined_pcd_folder)
o3d.io.write_point_cloud(f'{combined_pcd_folder}/{objectName}.ply', combined_pcd)
o3d.visualization.draw_geometries([combined_pcd])

Extracted 20 frames from the video.
Processing frame (480, 640)...
Points:
[[ 9.04261301e-03 -3.74234417e-02  2.72896215e+00]
 [ 1.39381302e-02 -5.70226171e-02  4.20637594e+00]
 [ 1.39381302e-02 -5.63613761e-02  4.20637594e+00]
 [ 1.39381302e-02 -5.57001352e-02  4.20637594e+00]
 [ 1.39381302e-02 -5.50388943e-02  4.20637594e+00]
 [ 1.39381302e-02 -5.43776534e-02  4.20637594e+00]
 [ 1.26971203e-02 -5.13975969e-02  4.02479624e+00]
 [ 1.26971203e-02 -5.07649003e-02  4.02479624e+00]
 [ 1.26971203e-02 -5.01322036e-02  4.02479624e+00]
 [ 1.26971203e-02 -4.94995070e-02  4.02479624e+00]
 [ 1.26971203e-02 -4.88668103e-02  4.02479624e+00]
 [ 1.26971203e-02 -4.82341137e-02  4.02479624e+00]
 [ 1.26971203e-02 -4.76014170e-02  4.02479624e+00]
 [ 1.26971203e-02 -4.69687204e-02  4.02479624e+00]
 [ 1.26971203e-02 -4.63360237e-02  4.02479624e+00]
 [ 1.26971203e-02 -4.57033271e-02  4.02479624e+00]
 [ 1.26971203e-02 -4.50706305e-02  4.02479624e+00]
 [ 1.26971203e-02 -4.44379338e-02  4.02479624e+00]
 [ 1.26

In [14]:
# Compute normals for the combined point cloud
print("Computing normals for the combined point cloud...")
combined_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Perform surface reconstruction using Poisson reconstruction
print("Performing surface reconstruction...")
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(combined_pcd, depth=9)

# Optionally, remove low-density vertices to clean up the mesh
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)

# Save and visualize the mesh
mesh_folder = f'./meshes/'
if not os.path.exists(mesh_folder):
    os.makedirs(mesh_folder)
o3d.io.write_triangle_mesh(f'{mesh_folder}/{objectName}.ply', mesh)
o3d.visualization.draw_geometries([mesh])

Computing normals for the combined point cloud...
Performing surface reconstruction...
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to m